In [2]:
# !pip install google.cloud
# !pip install --upgrade google-cloud-pubsub

In [1]:
import os
from google.cloud import pubsub_v1
from concurrent.futures import TimeoutError

In [25]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
timeout = 10

In [26]:
subscriber_obs = pubsub_v1.SubscriberClient()
subscription_path_obs = 'projects/weather-based-forecasting-v2/subscriptions/python_obs_sub'

subscriber_forecast = pubsub_v1.SubscriberClient()
subscription_path_forecast = 'projects/weather-based-forecasting-v2/subscriptions/python_forecast_sub'

In [27]:
def callback(message):
    print(f'Received message: {message}')
    print(f'data: {message.data[0:100]}')

    if message.attributes:
        print("Attributes:")
        for key in message.attributes:
            value = message.attributes.get(key)
            print(f"{key}: {value}")

    message.ack()

In [19]:
streaming_pull_future_obs = subscriber_obs.subscribe(subscription_path_obs, callback=callback)
print(f'Listening for messages on {subscription_path_obs}')

Listening for messages on projects/weather-based-forecasting-v2/subscriptions/python_obs_sub


In [28]:
streaming_pull_future_forecast = subscriber_forecast.subscribe(subscription_path_forecast, callback=callback)
print(f'Listening for messages on {subscription_path_forecast}')

Listening for messages on projects/weather-based-forecasting-v2/subscriptions/python_forecast_sub


In [3]:
with subscriber_obs:                                                # wrap subscriber in a 'with' block to automatically call close() when done
    try:
        # streaming_pull_future.result(timeout=timeout)
        streaming_pull_future_obs.result()                          # going without a timeout will wait & block indefinitely
    except TimeoutError:
        streaming_pull_future_obs.cancel()                          # trigger the shutdown
        streaming_pull_future_obs.result()                          # block until the shutdown is complete

In [4]:
with subscriber_forecast:                                                # wrap subscriber in a 'with' block to automatically call close() when done
    try:
        # streaming_pull_future.result(timeout=timeout)
        streaming_pull_future_forecast.result()                          # going without a timeout will wait & block indefinitely
    except TimeoutError:
        streaming_pull_future_forecast.cancel()                          # trigger the shutdown
        streaming_pull_future_forecast.result()                          # block until the shutdown is complete
